In [0]:
import torch
import numpy as np

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

In [0]:
cd drive/My Drive/DL_assignment/


In [0]:
pwd

In [0]:
cd ..


In [0]:
cd My Drive

In [0]:
cd DL_assignment/

In [0]:
from scipy.io import loadmat
train_data1 = loadmat('Vol_01_input.mat')
train_data2 = loadmat('Vol_02_input.mat')
train_data3 = loadmat('Vol_05_input.mat')

In [0]:
train_label1 = loadmat('Vol_01gt.mat')
train_label2 = loadmat('Vol_02gt.mat')
train_label3 = loadmat('Vol_05gt.mat')
# for keys in train_label1:
#   print(keys, train_label1[keys])

In [0]:
len(train_data1)

In [0]:
tr_data = train_data1['ana']
tr_data2 = train_data2['ana']
tr_data3 = train_data3['ana']
print(tr_data.shape[0])

In [0]:
tr_gt = train_label1['gt']
tr_gt2 = train_label2['gt']
tr_gt3 = train_label3['gt']

In [0]:
train_data = []
for i in range(tr_data.shape[2]):
  train_data.append(tr_data[:,][:,][i])
for i in range(tr_data.shape[2]):
  train_data.append(tr_data2[:,][:,][i])
for i in range(tr_data.shape[2]):
  train_data.append(tr_data3[:,][:,][i])
print(len(train_data))
print(train_data[0].shape)

In [0]:
train_gt = []
for i in range(tr_gt.shape[2]):
  train_gt.append(tr_gt[:,][:,][i])
for i in range(tr_gt.shape[2]):
  train_gt.append(tr_gt2[:,][:,][i])
for i in range(tr_gt.shape[2]):
  train_gt.append(tr_gt3[:,][:,][i])
print(len(train_gt))
print(train_gt[0].shape)

In [0]:
# print(train_gt[0] == 1)
def check(mat, st_i, st_j):
  i = st_i
  j = st_j
  flag = 0
  # print(i, j)
  while(i < st_i + 32):
    j = st_j
    while(j < st_j + 32):
      if mat[i][j] == 0:
        return False
      if mat[i][j] == 1:
        flag = 1
      j += 1
    i += 1
  # if flag == 1:
  #   return True
  # else:
  return True

def make_patch(mat, st_i, st_j):
  patch = np.zeros((32, 32))
  i = st_i
  j = st_j
  a = 0
  b = 0
  while(i < st_i + 32):
    j = st_j
    b = 0
    while(j < st_j + 32):
      patch[a][b] = mat[i][j]
      j += 1
      b += 1
    i += 1
    a += 1
  return patch





jump = 30
new_train_data = []
new_train_gt = []
for k in range(len(train_gt)):
  for i in range(tr_gt.shape[1] - 32):
    flag = 0
    for j in range(tr_gt.shape[2] - 32):
      if i != 0 and train_gt[k][i - 1][j] == 1:
        continue
      if train_gt[k][i][j] == 1:
        if check(train_gt[k], i, j) == True:
          patch_data = make_patch(train_data[k], i, j)
          patch_gt = make_patch(train_gt[k], i, j)
          new_train_data.append(patch_data)
          patch_gt = patch_gt - 1
          new_train_gt.append(patch_gt)
          j = j + jump
          flag = 1
    # if flag == 1:
    #   i = i + jump
          

print(len(new_train_data), len(new_train_gt))
print(new_train_data[12].shape, new_train_gt[12].shape)


In [0]:
for i in range(32):
  for j in range(32):
    print(new_train_gt[201][i][j], end = " ")
  print()


In [0]:
from matplotlib import pyplot as plt
plt.imshow(new_train_gt[103].astype('uint8'))
plt.show()

In [0]:
import torch
import numpy as np
from torch.utils import data



tensor_x = torch.Tensor(new_train_data) # transform to torch tensor
tensor_y = torch.Tensor(new_train_gt)

my_dataset = data.TensorDataset(tensor_x,tensor_y) # create your datset
my_dataloader = data.DataLoader(my_dataset) 

In [0]:
from torch import nn
import torch.nn.functional as F

class SegNet(nn.Module):
    
    def __init__(self):
        print("Hii")
        super(SegNet, self).__init__()
        print("Hello")
        self.conv1_1 = nn.Conv2d(1, 64, kernel_size = 3, stride = 1, padding = 1)
        self.conv1_2 = nn.Conv2d(64, 64, kernel_size = 3, stride = 1, padding = 1)
        self.conv2_1 = nn.Conv2d(64, 128, kernel_size = 3, stride = 1, padding = 1)
        self.conv2_2 = nn.Conv2d(128, 128, kernel_size = 3, stride = 1, padding = 1)
        self.conv3_1 = nn.Conv2d(128, 256, kernel_size = 3, stride = 1, padding = 1)
        self.conv3_2 = nn.Conv2d(256, 256, kernel_size = 3, stride = 1, padding = 1)
        self.conv3_3 = nn.Conv2d(256, 256, kernel_size = 3, stride = 1, padding = 1)
        self.conv4_1 = nn.Conv2d(256, 512, kernel_size = 3, stride = 1, padding = 1)
        self.conv4_2 = nn.Conv2d(512, 512, kernel_size = 3, stride = 1, padding = 1)
        self.conv4_3 = nn.Conv2d(512, 512, kernel_size = 3, stride = 1, padding = 1)
        self.conv5_1 = nn.Conv2d(512, 512, kernel_size = 3, stride = 1, padding = 1)
        self.conv5_2 = nn.Conv2d(512, 512, kernel_size = 3, stride = 1, padding = 1)
        self.conv5_3 = nn.Conv2d(512, 512, kernel_size = 3, stride = 1, padding = 1)

        self.pool1 = nn.MaxPool2d(kernel_size = 2, stride = 2, padding = 0, return_indices = True)
        self.pool2 = nn.MaxPool2d(kernel_size = 2, stride = 2, padding = 0, return_indices = True)
        self.pool3 = nn.MaxPool2d(kernel_size = 2, stride = 2, padding = 0, return_indices = True)
        self.pool4 = nn.MaxPool2d(kernel_size = 2, stride = 2, padding = 0, return_indices = True)
        self.pool5 = nn.MaxPool2d(kernel_size = 2, stride = 2, padding = 0, return_indices = True)
        
        self.unpool5 = nn.MaxUnpool2d(kernel_size = 2, stride = 2, padding = 0)
        self.unpool4 = nn.MaxUnpool2d(kernel_size = 2, stride = 2, padding = 0)
        self.unpool3 = nn.MaxUnpool2d(kernel_size = 2, stride = 2, padding = 0)
        self.unpool2 = nn.MaxUnpool2d(kernel_size = 2, stride = 2, padding = 0)
        self.unpool1 = nn.MaxUnpool2d(kernel_size = 2, stride = 2, padding = 0)

        self.conv_5_1 = nn.Conv2d(512, 512, kernel_size = 3, stride = 1, padding = 1)
        self.conv_5_2 = nn.Conv2d(512, 512, kernel_size = 3, stride = 1, padding = 1)
        self.conv_5_3 = nn.Conv2d(512, 512, kernel_size = 3, stride = 1, padding = 1)
        self.conv_4_1 = nn.Conv2d(512, 512, kernel_size = 3, stride = 1, padding = 1)
        self.conv_4_2 = nn.Conv2d(512, 512, kernel_size = 3, stride = 1, padding = 1)
        self.conv_4_3 = nn.Conv2d(512, 256, kernel_size = 3, stride = 1, padding = 1)
        self.conv_3_1 = nn.Conv2d(256, 256, kernel_size = 3, stride = 1, padding = 1)
        self.conv_3_2 = nn.Conv2d(256, 256, kernel_size = 3, stride = 1, padding = 1)
        self.conv_3_3 = nn.Conv2d(256, 128, kernel_size = 3, stride = 1, padding = 1)
        self.conv_2_1 = nn.Conv2d(128, 128, kernel_size = 3, stride = 1, padding = 1)
        self.conv_2_2 = nn.Conv2d(128, 64, kernel_size = 3, stride = 1, padding = 1)
        self.conv_1_1 = nn.Conv2d(64, 64, kernel_size = 3, stride = 1, padding = 1)
        self.conv_1_2 = nn.Conv2d(64, 1, kernel_size = 3, stride = 1, padding = 1)
        
        self.batch_norm1 = nn.BatchNorm2d(64)
        self.batch_norm2 = nn.BatchNorm2d(128)
        self.batch_norm3 = nn.BatchNorm2d(256)
        self.batch_norm4 = nn.BatchNorm2d(512)

    def forward(self, x):
        print("Alright?")
        size_1 = x.size()
        print("Here?")
        x = self.conv1_1(x)
        print("Reached Here?")
        x = self.batch_norm1(x)
        x = F.relu(x)
        x = self.conv1_2(x)
        x = self.batch_norm1(x)
        x = F.relu(x)
        x, idxs1 = self.pool1(x)
        
        size_2 = x.size()
        x = self.conv2_1(x)
        x = self.batch_norm2(x)
        x = F.relu(x)
        x = self.conv2_2(x)
        x = self.batch_norm2(x)
        x = F.relu(x)
        x, idxs2 = self.pool2(x)
        
        size_3 = x.size()
        x = self.conv3_1(x)
        x = self.batch_norm3(x)
        x = F.relu(x)
        x = self.conv3_2(x)
        x = self.batch_norm3(x)
        x = F.relu(x)
        x = self.conv3_3(x)
        x = self.batch_norm3(x)
        x = F.relu(x)
        x, idxs3 = self.pool3(x)
        
        size_4 = x.size()
        x = self.conv4_1(x)
        x = self.batch_norm4(x)
        x = F.relu(x)
        x = self.conv4_2(x)
        x = self.batch_norm4(x)
        x = F.relu(x)
        x = self.conv4_3(x)
        x = self.batch_norm4(x)
        x = F.relu(x)
        x, idxs4 = self.pool4(x)

        size_5 = x.size()
        x = self.conv5_1(x)
        x = self.batch_norm4(x)
        x = F.relu(x)
        x = self.conv5_2(x)
        x = self.batch_norm4(x)
        x = F.relu(x)
        x = self.conv5_3(x)
        x = self.batch_norm4(x)
        x = F.relu(x)
        x, idxs5 = self.pool5(x)

        
        x = self.unpool5(x, idxs5, output_size = size_5)
        x = self.conv_5_1(x)
        x = self.batch_norm4(x)
        x = F.relu(x)
        x = self.conv_5_2(x)
        x = self.batch_norm4(x)
        x = F.relu(x)
        x = self.conv_5_3(x)
        x = self.batch_norm4(x)
        x = F.relu(x)
        
        x = self.unpool4(x, idxs4, output_size = size_4)
        x = self.conv_4_1(x)
        x = self.batch_norm4(x)
        x = F.relu(x)
        x = self.conv_4_2(x)
        x = self.batch_norm4(x)
        x = F.relu(x)
        x = self.conv_4_3(x)
        x = self.batch_norm3(x)
        x = F.relu(x)
        
        x = self.unpool3(x, idxs3, output_size = size_3)
        x = self.conv_3_1(x)
        x = self.batch_norm3(x)
        x = F.relu(x)
        x = self.conv_3_2(x)
        x = self.batch_norm3(x)
        x = F.relu(x)
        x = self.conv_3_3(x)
        x = self.batch_norm2(x)
        x = F.relu(x)
        
        x = self.unpool2(x, idxs2, output_size = size_2)
        x = self.conv_2_1(x)
        x = self.batch_norm2(x)
        x = F.relu(x)
        x = self.conv_2_2(x)
        x = self.batch_norm1(x)
        x = F.relu(x)
        print("Hii")
        
        x = self.unpool1(x, idxs1, output_size = size_1)
        x = self.conv_1_1(x)
        x = self.batch_norm1(x)
        x = F.relu(x)
        x = self.conv_1_2(x)
        output = F.softmax(x, dim = 3)
        return x, output

def dice_loss(true, logits, eps=1e-7):
    """Computes the Sørensen–Dice loss.
    Note that PyTorch optimizers minimize a loss. In this
    case, we would like to maximize the dice loss so we
    return the negated dice loss.
    Args:
        true: a tensor of shape [B, 1, H, W].
        logits: a tensor of shape [B, C, H, W]. Corresponds to
            the raw output or logits of the model.
        eps: added to the denominator for numerical stability.
    Returns:
        dice_loss: the Sørensen–Dice loss.
    """
    num_classes = logits.shape[1]
    print(num_classes)
    if num_classes == 1:
        true_1_hot = torch.eye(num_classes + 1)[true.squeeze(1)]
        true_1_hot = true_1_hot.permute(0, 3, 1, 2).float()
        true_1_hot_f = true_1_hot[:, 0:1, :, :]
        true_1_hot_s = true_1_hot[:, 1:2, :, :]
        true_1_hot = torch.cat([true_1_hot_s, true_1_hot_f], dim=1)
        pos_prob = torch.sigmoid(logits)
        neg_prob = 1 - pos_prob
        probas = torch.cat([pos_prob, neg_prob], dim=1)
    else:
        true_1_hot = torch.eye(num_classes)[true.squeeze(1)]
        true_1_hot = true_1_hot.permute(0, 3, 1, 2).float()
        probas = F.softmax(logits, dim=1)
    true_1_hot = true_1_hot.type(logits.type())
    dims = (0,) + tuple(range(2, true.ndimension()))
    intersection = torch.sum(probas * true_1_hot, dims)
    cardinality = torch.sum(probas + true_1_hot, dims)
    dice_loss = (2. * intersection / (cardinality + eps)).mean()
    # print(dice_loss, 1 - dice_loss)
    return (dice_loss)

# def confusion_matrix(output, label):
#   output = np.array(output)
#   label = np.array(label)
#   for i in range(output):



In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
learning_rate = 0.001
epochs = 1
model = SegNet()
if torch.cuda.is_available():
  model.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate, momentum = 0.9, weight_decay = 0.005)
        

In [0]:
if torch.cuda.is_available():
  print('true')
else:
  print('false')

In [0]:
import time

for i in range(epochs):
        start_time = time.time()
        mean_loss = 0
        tot = 0
        idx = 0
        for img, label in my_dataloader:
            if torch.cuda.is_available():
              label = label.cuda()
              img = img.cuda()
            img_r = torch.unsqueeze(img, 1)
            img = img.type(torch.FloatTensor)
            label_r = torch.unsqueeze(label, 1)
            label = label.type(torch.LongTensor)
            label = label.cuda()
            optimizer.zero_grad()
            output, output_s = model(img_r)
            # print(output.shape, label.shape)
            loss = criterion(output, label)
            loss2 = dice_loss(label, output)
            # print(loss2.item())
            tot += loss2.item()
            loss.backward()
            optimizer.step()
            mean_loss += loss
            idx += 1

        print(idx)  
        mean_loss /= idx
        tot /= idx
        end_time = time.time()
        elapse_time = end_time - start_time
        print(f'epoch {i} loss: {mean_loss}, elapse time: {elapse_time}', tot)
        # if mean_loss < min_loss:
        #     print(f'in epoch {i}, loss decline')
        #     min_loss = mean_loss
        #     state_dict = model.module.state_dict()
        #     for key in state_dict.keys():
        #         state_dict[key] = state_dict[key].cpu()
        #         torch.save(state_dict, 'segnet_weight_11classes.pth')

In [0]:
output.shape

In [0]:
x1 = output.argmax()

In [0]:
x = output.detach().cpu()
x = np.array(x)

In [0]:
x.shape

In [0]:
x[0][2][i][j]

In [0]:
x1 = x1.detach().cpu()

In [0]:
x1 = np.array(x1)
print(x1.shape)

In [0]:
x1

In [0]:
pip install torchsummary

In [0]:
from torchsummary import summary
summary(model,(1,1,32,32))